## <font color=yellow>1. Chroma DB</font>

In [1]:
import pandas as pd

from tqdm import tqdm
from datasets import load_dataset

import chromadb

#chroma_client = chromadb.PersistentClient()
chroma_client = chromadb.Client()

# Create a collection (나만의 고유한 DB를 만듦)
#chroma_client.delete_collection(name="my_collection")
collection = chroma_client.create_collection(name="my_collection")

In [14]:
df = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train[:20]').to_pandas()
df.head()

,date,category,press,title,document,link,summary
0,2022-07-03 17:14:37,economy,YTN,추경호 중기 수출지원 총력 무역금융 40조 확대,앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 ...,https://n.news.naver.com/mnews/article/052/000...,"올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, ..."
1,2022-07-04 08:07:12,economy,아시아경제,해산물·주류 무제한 인터컨티넨탈 뷔페 여름 한정 페스타 연다,문어 랍스터 대게 갑오징어 새우 소라 등 해산물 활용 미국식 해물찜 시푸드 보일 준...,https://n.news.naver.com/mnews/article/277/000...,인터엑스 1층 뷔페 레스토랑 브래서리는 오는 6일부터 8월31일까지 쿨 섬머 페스타...
2,2022-07-05 18:01:01,IT과학,동아사이언스,한국인 생활 변화시킨 의사 마지막까지 영향력 컸던 과학자 이호왕 고려대 명예교수 소천,한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학...,https://n.news.naver.com/mnews/article/584/000...,이 이호왕 고려대 명예교수는 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을...
3,2022-07-01 08:51:12,economy,뉴시스,에디슨이노 이승훈 제이스페이스 대표 사내이사 선임,기사내용 요약 우주발사체 사업 본격화 서울 뉴시스 김경택 기자 에디슨이노가 우주발사...,https://n.news.naver.com/mnews/article/003/001...,에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이...
4,2022-07-01 16:11:01,economy,머니투데이,SK바사 해외 사업 조직 개편… 글로벌 탑티어 기업으로 성장 박차,SK바이오사이언스가 글로벌 사업의 고도화를 위해 조직 개편을 단행했다. SK바이오사...,https://n.news.naver.com/mnews/article/008/000...,SK바이오사이언스가 글로벌 사업의 고도화를 위해 기존 해외사업개발실을 백신사업뿐만 ...


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('snunlp/KR-SBERT-V40K-KlueNLI-augSTS')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\jjsd4\workspace\torch\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jjsd4\.cache\huggingface\hub\models--snunlp--KR-SBERT-V40K-KlueNLI-augSTS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pandas에 들어가있는 데이터들을 블록으로 나눌 것임
- 데이터 많은 양을 그냥 넣으면, db에 들어갔을때 유사성 검색 이런 것 하기가 어려워짐
- 검색하기 용이하고, 많은 용량의 문서 파일에서 내가 질문 한 것에 최적화되어 있는 데이터를 만들어주길 원함
- context로 들어가는 문서들을 조각조각 블록화시켜야함

In [4]:
# 데이터 넣기

ids = []
metadatas = []
embeddings = []

#date	category	press	title	document	link	summary

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].title
    answer = row[1].summary

    metadata = {
        "query": query,
        "answer": answer
    }

    embedding = model.encode(query, normalize_embeddings=True)

    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)

chunk_size = 1024  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(embeddings) // chunk_size + 1  # 전체 데이터를 chunk 단위로 나눈 횟수
embeddings = [e.tolist() for e in tqdm(embeddings)]

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size

    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_ids = ids[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]

    # chunk를 answers에 추가
    collection.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

20it [00:01, 19.63it/s]
100%|██████████| 1/1 [00:00<00:00, 13.70it/s]


pdf에서 문장을 검색할때, 블록으로 자르면 갭이 생김
- 오버랩되게 잘라야함

In [ ]:
result = collection.query(
    query_embeddings=model.encode("미국 뉴욕증시 52년만에", normalize_embeddings=True).tolist(),
    n_results=3
    )

print(result)

{'ids': [['11', '3', '15']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'query': '특징주 삼성전자·SK하이닉스 또다시 52주 신저가 추락', 'answer': '4일 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에, SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래되면서 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다.'}, {'answer': '에디슨이노는 1일 임시주주총회를 통해 사명을 이노시스 로 변경하고 이승훈 제이스페이스홀딩스 대표이사를 사내이사로 선임하고 위성체 발사와 우주선 위성시스템 등 항공 우주 분야와 자율 주행·그래핀 관련 사업을 사업목적에 추가하여 우주발사체 사업에 본격 진출한다고 1일 밝혔다.', 'query': '에디슨이노 이승훈 제이스페이스 대표 사내이사 선임'}, {'answer': '삼성바이오로직스는 4일 공시를 통해 미국 제약기업 MSD MSD International Business GmbH 와 2768억2938만원 규모의 의약품 위탁생산 공급계약을 2022년 7월 1일부터 2028년 12월 31일까지 최근 매출액 대비 17.65% 규모로 체결했다고 밝혔다.', 'query': '삼성바이오로직스 MSD와 2768억원 위탁생산계약 체결'}]], 'distances': [[0.8033769130706787, 1.4475741386413574, 1.4505667686462402]]}


## <font color=yellow>2. Chroma DB with LangChain</font>

In [9]:
import os 

from langchain_chroma import Chroma
# from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema import Document

if not os.path.exists("naver_news.csv"):
    dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train[:20]')
    dataset.to_csv("naver_news.csv", index=False)

loader = CSVLoader("naver_news.csv", encoding='utf-8')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)

query = "에디슨이노 이승훈"
docs = db.similarity_search(query)

print(docs[0].page_content)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_2788\2168574625.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\jjsd4\workspace\torch\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jjsd4\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

date: 2022-07-05 18:42:03
category: economy
press: 연합뉴스TV
title: 과기부 허준이 필즈상 기쁜 소식…한국 수학 발전 기대
document: 정부가 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했습니다. 과학기술정보통신부는 대수기하학의 토대가 더욱 확장되도록 새 지평을 연 공로를 인정받아 한국계 최초로 필즈상 수상의 쾌거를 이룩했다 며 축하했습니다. 또 허 교수의 필즈상 수상은 올해 국제수학연맹에서 한국의 수학 국가등급을 최고등급으로 상향한 것에 이은 기쁜 소식 이라며 허 교수의 행보와 한국 수학계의 지속적 발전 역시 기대된다고 덧붙였습니다.
link: https://n.news.naver.com/mnews/article/422/0000549728?sid=101
summary: 정부가 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했고 과학기술정보통신부는 대수기하학의 토대가 더욱 확장되도록 새 지평을 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 공로를 연 허준이 미국 프린스턴대 교수 겸 한국 고등과학원 수학부 석학교수의 필즈상 수상을 축하했다.


In [11]:
documents

[Document(metadata={'source': 'naver_news.csv', 'row': 0}, page_content='date: 2022-07-03 17:14:37\ncategory: economy\npress: YTN\ntitle: 추경호 중기 수출지원 총력 무역금융 40조 확대\ndocument: 앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 

## <font color=yellow>3. FAISS</font>

In [1]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 데이터 불러오기
df = pd.read_csv("test.csv")
df.head()

,date,category,press,title,document,link,summary
0,2022-07-04 08:47:54,economy,머니투데이,아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원,아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출...,https://n.news.naver.com/mnews/article/008/000...,"아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 ..."
1,2022-07-07 15:01:04,IT과학,파이낸셜뉴스,뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든다,한국형 우주발사체 누리호 가 지난 6월 21일 전남 고흥 나로우주센터 발사대에서 날...,https://n.news.naver.com/mnews/article/014/000...,한국형 우주발사체 누리호가 지난 6월 21일 전남 고흥 나로우주센터 발사대에서 성공...
2,2022-07-01 06:08:01,economy,매일경제,SP500 올 상반기 21%↓…50년래 최악 월가월부,닥터둠 루비니 주가 50% 폭락할 것 뉴욕 증시 3대지수 모두 하락 대형주 중심의 ...,https://n.news.naver.com/mnews/article/009/000...,대 대형주 중심의 미국 스탠더드앤드푸어스 S P 500 지수가 올 상반기 50여년만...
3,2022-07-05 12:29:02,economy,파이낸셜뉴스,풀무원 알래스칸 명태살 ‘볼카츠’ 출시,파이낸셜뉴스 풀무원식품은 알래스카 청정해역의 신선한 명태살로 만든 연육을 바삭하게 ...,https://n.news.naver.com/mnews/article/014/000...,5일 5일 풀무원식품은 알래스카 청정해역의 신선한 명태살로 만든 연육을 동그랗게 빚...
4,2022-07-03 10:49:06,IT과학,디지털타임스,일동제약 생산본부장에 강덕원 부사장 영입,강덕원 일동제약 생산본부장. 일동제약 제공 일동제약은 생산본부장으로 강덕원 부사장을...,https://n.news.naver.com/mnews/article/029/000...,일동제약은 생산본부장으로 강덕원 부사장을 영입했다고 지난 1일 밝혔으며 강덕원 본부...


In [2]:
texts = df.to_numpy()
texts

array([['2022-07-04 08:47:54', 'economy', '머니투데이 ', ...,
        '아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.',
        'https://n.news.naver.com/mnews/article/008/0004766307?sid=101',
        '아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔으며 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것으로, 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다.'],
       ['2022-07-07 15:01:04', 'IT과학', '파이낸셜뉴스 ', ...,
        '한국형 우

In [3]:
texts.shape

(2740, 7)

In [4]:
for idx, content in enumerate(texts):
    if idx == 2:
        break
    print(content)

['2022-07-04 08:47:54' 'economy' '머니투데이 '
 '아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원'
 '아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.'
 'https://n.news.naver.com/mnews/article/008/0004766307?sid=101'
 '아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔으며 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것으로, 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다.']
['2022-07-07 15:01:04' 'IT과학' '파이낸셜뉴스 ' '뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든

In [5]:
for idx, content in enumerate(texts):
    if idx == 2:
        break
    print(content[3])

아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원
뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든다


In [6]:
titles = [item[3] for item in texts]
embedder = SentenceTransformer('Huffon/sentence-klue-roberta-base')
vectors = embedder.encode(titles, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
print(vectors.shape)

No sentence-transformers model found with name Huffon/sentence-klue-roberta-base. Creating a new one with mean pooling.


Batches:   0%|          | 0/86 [00:00<?, ?it/s]

(2740, 768)


In [7]:
# Faiss 계산하기
# 초기화 : 벡터의 크기를 지정
#(IndexFlatL2는 각 벡터의 유사성 또는 각 벡터의 근접성을 측정하는 거리 메트릭)
# L2는 유클리드 거리를 의미
index = faiss.IndexFlatIP(vectors.shape[1])
index.add(vectors) # 임베딩 추가

In [8]:
index.is_trained #추가 되었는지 확인
index.ntotal #추가된 벡터의 수

2740

In [9]:
## 유사도 검색

top_k = 4
query = "세입자 구해오라는 집주인"
query_embedding = embedder.encode(query, normalize_embeddings=True, convert_to_numpy=True)
distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k=top_k)

# 유사도 검색은 쉽게 index.search를 사용하면 할수 있음
# 결과는 튜플형태로 나오게 되는데 앞에(distances)는 각각의 벡터들의 거리가 저장되고,
# 뒤에 (indices)는 해당하는 인덱스를 반화
# top_k는 몇개를 추출할 것인지에 대한 옵션이고 자동으로 거리가 가까운 순서대로 정렬되어 반환됨

In [10]:
# 결과 확인
temp = df.iloc[indices[0]]
temp['distance'] = distances[0]
temp[['title', 'document', 'link', 'summary']]

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_34236\3202029093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['distance'] = distances[0]


,title,document,link,summary
812,전세금 받고 싶으면 세입자 구해오라는 집주인 대처법더 머니이스트아하 부동산법률,한경닷컴 더 머니이스트 중도 해지 시 집주인 요구 정당 묵시적 갱신 · 갱신 요구권...,https://n.news.naver.com/mnews/article/015/000...,"신규 세입자를 구하는 일을 두고 집주인과 세입자 간 갈등이 생기는 경우가 있는데, ..."
2396,전셋값 오르고 이자 뛰고..‘차라리 월세 살래요’,3일 서울 송파구에 위치한 한 공인중개사 사무소에 아파트 매매 및 전·월세 가격표가...,https://n.news.naver.com/mnews/article/005/000...,최근 전·월세 거래 중 월세 비중이 지난 4월 통계 집계 이후 처음 50%를 넘었고...
1681,서울 아파트 매수심리 8주째 하락…전세도 ‘공급수요’,서울 아파트 매수심리가 8주 연속 하락한 것으로 조사됐습니다. 한국부동산원은 이번 ...,https://n.news.naver.com/mnews/article/056/001...,한국부동산원은 이번 주 서울 아파트 매매수급지수가 87.0으로 지난주보다 1.1포인...
2227,팔 사람은 많고 살 사람은 없다…수도권 아파트 수급지수 뚝뚝,수도권 매매수급지수 3년여만에 90 아래로 서울 8주째 매수 위축…거래절벽·가격하락...,https://n.news.naver.com/mnews/article/277/000...,전국 한국부동산원 조사에 따르면 이번주 수도권 아파트 매매수급지수는 89.8을 기록...
